<a href="https://colab.research.google.com/github/Paarmenid/Primjena-strojnog-u-enja-za-korekciju-artefakata-u-fluorescencijskoj-korelacijskoj-spektroskopiji/blob/main/Diplomski_kod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install lmfit

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import random
from tensorflow.keras.layers import Dropout
from google.colab import drive
import lmfit
from lmfit import Model
import os

In [ ]:
!pip install ipython-autotime

%load_ext autotime



In [ ]:
drive.mount("/content/gdrive")

In [ ]:
# Direktorij u kojemu se nalaze podaci po danima mjerenja
#cd "/content/gdrive/My Drive/Diplomski"

path=r"/content/gdrive/My Drive/Diplomski/mjerenja"


# Funkcije

In [ ]:
def read_data(podskup_mjerenja,i,folder):
    
 
 if podskup_mjerenja<10: 
    if i>9:
        p=path+"/"+str(folder)
        a=pd.read_csv(p+"/"+str(podskup_mjerenja)+"_C"+str(i)+".ASC",sep="\t",encoding= 'unicode_escape')
        a=a[11:]
        a=a[:-1]
        a[["Time[µs]","FCS_value"]]=a[a.columns[0]].str.split(expand=True)
        a=a.drop(["  Title : "+str(podskup_mjerenja)+"_c"+str(i)],axis=1)
        a=a.rename(columns={a.columns[1]:"G_"+str(podskup_mjerenja)+"_"+str(i)})
        a=a.astype(float)
    else:
        p=path+"/"+str(folder)
        a=pd.read_csv(p+"/"+str(podskup_mjerenja)+"_C"+str(i)+".ASC",sep="\t",encoding= 'unicode_escape')
        a=a[11:]
        a=a[:-1]
        a[["Time[µs]","FCS_value"]]=a[a.columns[0]].str.split(expand=True)
        a=a.drop(["  Title : "+str(podskup_mjerenja)+"_c0"+str(i)],axis=1)
        a=a.rename(columns={a.columns[1]:"G_"+str(podskup_mjerenja)+"_0"+str(i)})
        a=a.astype(float)
        
 else:
    if i>9:
        p=path+"/"+str(folder)
        a=pd.read_csv(p+"/"+str(podskup_mjerenja)+"_C"+str(i)+".ASC",sep="\t",encoding= 'unicode_escape')
        a=a[11:]
        a=a[:-1]
        a[["Time[µs]","FCS_value"]]=a[a.columns[0]].str.split(expand=True)
        a=a.drop(["  Title : "+str(podskup_mjerenja)+"_c"+str(i)],axis=1)
        a=a.rename(columns={a.columns[1]:"G_"+str(podskup_mjerenja)+"_"+str(i)})
        a=a.astype(float)
    else:
        p=path+"/"+str(folder)
        a=pd.read_csv(p+"/"+str(podskup_mjerenja)+"_C"+str(i)+".ASC",sep="\t",encoding= 'unicode_escape')
        a=a[11:]
        a=a[:-1]
        a[["Time[µs]","FCS_value"]]=a[a.columns[0]].str.split(expand=True)
        a=a.drop(["  Title : "+str(podskup_mjerenja)+"_c0"+str(i)],axis=1)
        a=a.rename(columns={a.columns[1]:"G_"+str(podskup_mjerenja)+"_0"+str(i)})
        a=a.astype(float)
    
 return a
def mjerenje(podskup_mjerenja,folder):
    start=2
    
    
    try:
        df=read_data(podskup_mjerenja,1,folder)  
    except:
        try:
            df=read_data(podskup_mjerenja,2,folder)  
            start=3
        except:
            try:
                df=read_data(podskup_mjerenja,3,folder)  
                start=4
            except:
                try:
                    df=read_data(podskup_mjerenja,4,folder)  
                    start=5
                except:
                    try:
                         df=read_data(podskup_mjerenja,5,folder)  
                         start=6
                    except:
                        try:
                            df=read_data(podskup_mjerenja,6,folder)  
                            start=7
                        except:
                            try:
                                df=read_data(podskup_mjerenja,7,folder)  
                                start=8
                            except:
                                try:
                                    df=read_data(podskup_mjerenja,8,folder)  
                                    start=9
                                except:
                                    try:
                                        df=read_data(podskup_mjerenja,9,folder)  
                                        start=10
                                    except:
                                        try:
                                            df=read_data(podskup_mjerenja,10,folder)  
                                            start=11
                                        except:
                                            try:
                                                df=read_data(podskup_mjerenja,11,folder)  
                                                start=12
                                            except:
                                                try:
                                                    df=read_data(podskup_mjerenja,12,folder)  
                                                    start=13
                                                except:
                                                    try:
                                                        df=read_data(podskup_mjerenja,13,folder)  
                                                        start=14
                                                    except:
                                                        df=read_data(podskup_mjerenja,14,folder)  
                                                        start=15
                                                    
        
    for i in range(start,16):
        try:
            a=read_data(podskup_mjerenja,i,folder)
            #print(a.columns)
            df=pd.merge(df,a,how="right",on="Time[µs]")
        except:
                print("ne postoji mjerenje"+str(i))
    return df

def pivot(df):
    df=pd.pivot_table(df,values=list(df.columns[1:]),columns="Time[µs]")
    return df


def skaliranje_unutar_seta(df):
    #pomicanje mjerenja unutar seta mjerenja u odnosu na jedno mjerenje(referentno)

    for i in range(1,len(df.columns)):
        faktor_skaliranja=df.iloc[100:130,1].mean()/df.iloc[100:130,i].mean() 
        df.iloc[:,i]=df.iloc[:,i]*faktor_skaliranja
    return df   



def skaliranje(df1,df2):
    #skaliranje seta mjerenja prema referentnom setu df3_1

    return df1.iloc[75:90].mean().mean()/df2.iloc[70:90].mean().mean()


def mjerenje_transf(broj_mjerenja,datum):
    #postavljanje vremena kao indeks varijable + log10 vremena

    df=mjerenje(broj_mjerenja,datum)
    df=df.set_index("Time[µs]")
    
    return df



def NN():


    rand=random.randint(1,1000)

    keras.backend.clear_session()
    np.random.seed(rand)
    tf.random.set_seed(rand)
        
    model = keras.models.Sequential()
  
    model.add(keras.layers.Dense(50, activation="relu"))
    keras.layers.Dropout(rate=0.2)

    model.add(keras.layers.Dense(20, activation="relu"))
    keras.layers.Dropout(rate=0.2)
    
    model.add(keras.layers.Dense(1))

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-3,
        decay_steps=100000,
        decay_rate=0.90)

    opt = keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.90, beta_2=0.999)
        
    model.compile(loss="mean_squared_error", optimizer=opt)

    early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss",patience=10,
                                                      restore_best_weights=True)

    checkpoint_cb = [
        keras.callbacks.ModelCheckpoint(
            filepath="model_2.keras",
            save_best_only=True,
            monitor="val_loss")
    ]

    history = model.fit(np.array(train.iloc[:,10:-1]), np.array(train.iloc[:,-1]), epochs=200, validation_data=(np.array(valid.iloc[:,10:-1]), np.array(valid.iloc[:,-1])),callbacks=[checkpoint_cb, early_stopping_cb])


    model = keras.models.load_model(
        "model_2.keras")
    return model


def rezultati():
   
    test_mean=[]
    test_std=[]
    test_rel_error=[]

    for i in range(7):

      D_lab,D_pred=test["D"][granice[i]:granice[i+1]],test["D_pred"][granice[i]:granice[i+1]]
      n=len(D_pred)
      mean=sum(D_pred)/n
      razlika=(D_pred-mean)**2
      suma=sum(razlika)

      test_mean.append(np.round(mean,3))
      test_std.append(np.round(np.sqrt(suma/((n-1)*n)),4))

    return test_mean,test_std

def run_vise_modela(broj_runova):

  train,valid=train_test_split(df,test_size=0.20,random_state=random.randint(1,1000),shuffle=True)

  ukupni_NN_mean=[]
  ukupni_NN_std=[]


  for i in range(broj_runova):
    
    print("Run broj:",i)

    train,valid=train_test_split(df,test_size=0.2,random_state=random.randint(1,1000),shuffle=True)

    model=NN()

    if len(test.columns)==255:
             y_pred = model.predict(np.array(test.iloc[:,10:-1]))
    else:
             y_pred = model.predict(np.array(test.iloc[:,10:-2]))

    test["D_pred"]=y_pred

    test_mean,test_std=rezultati()

    ukupni_NN_mean.append(test_mean)
    ukupni_NN_std.append(test_std)

  print (model.summary())
  return np.array(ukupni_NN_mean),np.array(ukupni_NN_std)


def FCS_autokorelacija(tau,tau_trip,tau_D,G_0,T):
    
   return ((1-T+T*np.exp(-tau/tau_trip))/(1-T))*G_0*(1/(1+(tau/tau_D)))*(1/np.sqrt(1+((1/5)**2)*(tau/tau_D)))


def fit_test(test_set,D_true):
    
    D_lista_test=[]
    M_D_lista_test=[]
    D_razlika=[]
    
    for i,df in enumerate(test_set):
        
            tau_D_lista=[]
            M_tau_D_lista=[]
            D_lista=[]
            M_D_lista=[]

            pocetak=10
            tau=x[pocetak:]
            for j in range(0,len(df.columns)):
                y=df.iloc[pocetak:,j].values
                result = autokor.fit(y, tau=tau, G_0=0.3, tau_D=127, tau_trip=1,T=0.85)
                
                t_D=result.params["tau_D"].value
                M_tD=result.params["tau_D"].stderr

                D=r_0**2/(4*t_D*1E-6)
                M_D=(r_0**2/(4*(t_D*1E-6)**2))*(M_tD*1E-6)
                
                tau_D_lista.append(t_D)
                M_tau_D_lista.append(M_tD)
                D_lista.append(D)
                M_D_lista.append(M_D)

            sum1=sum([D_lista[i]/(M_D_lista[i]**2) for i in range(len(D_lista))])
            sum2=sum([1/(M_D_lista[i]**2) for i in range(len(M_D_lista))])
            opca_srednja_vrijednost=sum1/sum2
            
            D_lista_test.append(opca_srednja_vrijednost)
            M_D_lista_test.append(np.sqrt(1/sum2))
            D_razlika.append(abs(D_true[i]-opca_srednja_vrijednost))
            
    return list(np.round(D_lista_test,13)),list(np.round(M_D_lista_test,15)),list(np.round(D_razlika,14))

# Učitavanje podataka

In [ ]:
df3_1=mjerenje_transf(1,"19.9 difuzija")
df3_2=mjerenje_transf(2,"19.9 difuzija")
df3_3=mjerenje_transf(3,"19.9 difuzija")
df3_4=mjerenje_transf(4,"19.9 difuzija")
df3_5=mjerenje_transf(5,"19.9 difuzija")
df3_6=mjerenje_transf(6,"19.9 difuzija")
df3_7=mjerenje_transf(7,"19.9 difuzija")
df3_8=mjerenje_transf(8,"19.9 difuzija")
df3_9=mjerenje_transf(9,"19.9 difuzija")
df3_10=mjerenje_transf(10,"19.9 difuzija")
df3_11=mjerenje_transf(11,"19.9 difuzija")
df3_12=mjerenje_transf(12,"19.9 difuzija")
df3_13=mjerenje_transf(13,"19.9 difuzija")
df3_14=mjerenje_transf(14,"19.9 difuzija")
df3_15=mjerenje_transf(15,"19.9 difuzija")
df3_16=mjerenje_transf(16,"19.9 difuzija")
df3_17=mjerenje_transf(17,"19.9 difuzija")
df3_18=mjerenje_transf(18,"19.9 difuzija")

df3_1=df3_1-df3_1.iloc[50:].min()
df3_2=df3_2-df3_2.iloc[50:].min()
df3_3=df3_3-df3_3.iloc[50:].min()
df3_4=df3_4-df3_4.iloc[50:].min()
df3_5=df3_5-df3_5.iloc[50:].min()
df3_6=df3_6-df3_6.iloc[50:].min()
df3_7=df3_7-df3_7.iloc[50:].min()
df3_8=df3_8-df3_8.iloc[50:].min()
df3_9=df3_9-df3_9.iloc[50:].min()
df3_10=df3_10-df3_10.iloc[50:].min()
df3_11=df3_11-df3_11.iloc[50:].min()
df3_12=df3_12-df3_12.iloc[50:].min()
df3_13=df3_13-df3_13.iloc[50:].min()
df3_14=df3_14-df3_14.iloc[50:].min()
df3_15=df3_15-df3_15.iloc[50:].min()
df3_16=df3_16-df3_16.iloc[50:].min()
df3_17=df3_17-df3_17.iloc[50:].min()
df3_18=df3_18-df3_18.iloc[50:].min()

df5_20=mjerenje_transf(20,"22.9 difuzija")
df5_21=mjerenje_transf(21,"22.9 difuzija")
df5_22=mjerenje_transf(22,"22.9 difuzija")
df5_23=mjerenje_transf(23,"22.9 difuzija")

df5_20=df5_20-df5_20.iloc[50:].min()
df5_21=df5_21-df5_21.iloc[50:].min()
df5_22=df5_22-df5_22.iloc[50:].min()
df5_23=df5_23-df5_23.iloc[50:].min()


df6_1=mjerenje_transf(1,"23.9 difuzija")
df6_2=mjerenje_transf(2,"23.9 difuzija")
df6_3=mjerenje_transf(3,"23.9 difuzija")
df6_4=mjerenje_transf(4,"23.9 difuzija")
df6_5=mjerenje_transf(5,"23.9 difuzija")
df6_6=mjerenje_transf(6,"23.9 difuzija")
df6_7=mjerenje_transf(7,"23.9 difuzija")
df6_8=mjerenje_transf(8,"23.9 difuzija")
df6_9=mjerenje_transf(9,"23.9 difuzija")
df6_10=mjerenje_transf(10,"23.9 difuzija")
df6_11=mjerenje_transf(11,"23.9 difuzija")
df6_12=mjerenje_transf(12,"23.9 difuzija")
df6_13=mjerenje_transf(13,"23.9 difuzija")
df6_14=mjerenje_transf(14,"23.9 difuzija")
df6_15=mjerenje_transf(15,"23.9 difuzija")

df6_1=df6_1-df6_1.iloc[50:].min()
df6_2=df6_2-df6_2.iloc[50:].min()
df6_3=df6_3-df6_3.iloc[50:].min()
df6_4=df6_4-df6_4.iloc[50:].min()
df6_5=df6_5-df6_5.iloc[50:].min()
df6_6=df6_6-df6_6.iloc[50:].min()
df6_7=df6_7-df6_7.iloc[50:].min()
df6_8=df6_8-df6_8.iloc[50:].min()
df6_9=df6_9-df6_9.iloc[50:].min()
df6_10=df6_10-df6_10.iloc[50:].min()
df6_11=df6_11-df6_11.iloc[50:].min()
df6_12=df6_12-df6_12.iloc[50:].min()
df6_13=df6_13-df6_13.iloc[50:].min()
df6_14=df6_14-df6_14.iloc[50:].min()
df6_15=df6_15-df6_15.iloc[50:].min()


df7_1=mjerenje_transf(1,"24.9 difuzija")
df7_2=mjerenje_transf(2,"24.9 difuzija")
df7_3=mjerenje_transf(3,"24.9 difuzija")
df7_4=mjerenje_transf(4,"24.9 difuzija")
df7_5=mjerenje_transf(5,"24.9 difuzija")
df7_6=mjerenje_transf(6,"24.9 difuzija")
df7_7=mjerenje_transf(7,"24.9 difuzija")
df7_8=mjerenje_transf(8,"24.9 difuzija")
df7_9=mjerenje_transf(9,"24.9 difuzija")
df7_10=mjerenje_transf(10,"24.9 difuzija")
df7_11=mjerenje_transf(11,"24.9 difuzija")
df7_12=mjerenje_transf(12,"24.9 difuzija")
df7_13=mjerenje_transf(13,"24.9 difuzija")

df7_1=df7_1-df7_1.iloc[50:].min()
df7_2=df7_2-df7_2.iloc[50:].min()
df7_3=df7_3-df7_3.iloc[50:].min()
df7_4=df7_4-df7_4.iloc[50:].min()
df7_5=df7_5-df7_5.iloc[50:].min()
df7_6=df7_6-df7_6.iloc[50:].min()
df7_7=df7_7-df7_7.iloc[50:].min()
df7_8=df7_8-df7_8.iloc[50:].min()
df7_9=df7_9-df7_9.iloc[50:].min()
df7_10=df7_10-df7_10.iloc[50:].min()
df7_11=df7_11-df7_11.iloc[50:].min()
df7_12=df7_12-df7_12.iloc[50:].min()
df7_13=df7_13-df7_13.iloc[50:].min()


df8_1=mjerenje_transf(1,"25.9 difuzija")
df8_2=mjerenje_transf(2,"25.9 difuzija")
df8_3=mjerenje_transf(3,"25.9 difuzija")
df8_4=mjerenje_transf(4,"25.9 difuzija")
df8_5=mjerenje_transf(5,"25.9 difuzija")
df8_6=mjerenje_transf(6,"25.9 difuzija")
df8_7=mjerenje_transf(7,"25.9 difuzija")

df8_1=df8_1-df8_1.iloc[50:].min()
df8_2=df8_2-df8_2.iloc[50:].min()
df8_3=df8_3-df8_3.iloc[50:].min()
df8_4=df8_4-df8_4.iloc[50:].min()
df8_5=df8_5-df8_5.iloc[50:].min()
df8_6=df8_6-df8_6.iloc[50:].min()
df8_7=df8_7-df8_7.iloc[50:].min()



#df3_1=skaliranje_unutar_seta(df3_1)
df3_2=skaliranje_unutar_seta(df3_2)
df3_3=skaliranje_unutar_seta(df3_3)
df3_4=skaliranje_unutar_seta(df3_4)
df3_5=skaliranje_unutar_seta(df3_5)
df3_6=skaliranje_unutar_seta(df3_6)
df3_7=skaliranje_unutar_seta(df3_7)
df3_8=skaliranje_unutar_seta(df3_8)
df3_9=skaliranje_unutar_seta(df3_9)
df3_10=skaliranje_unutar_seta(df3_10)
df3_11=skaliranje_unutar_seta(df3_11)
df3_12=skaliranje_unutar_seta(df3_12)
df3_13=skaliranje_unutar_seta(df3_13)
df3_14=skaliranje_unutar_seta(df3_14)
df3_15=skaliranje_unutar_seta(df3_15)
df3_16=skaliranje_unutar_seta(df3_16)
df3_17=skaliranje_unutar_seta(df3_17)
df3_18=skaliranje_unutar_seta(df3_18)

df5_20=skaliranje_unutar_seta(df5_20)
df5_21=skaliranje_unutar_seta(df5_21)
df5_22=skaliranje_unutar_seta(df5_22)
df5_23=skaliranje_unutar_seta(df5_23)

df6_1=skaliranje_unutar_seta(df6_1)
df6_2=skaliranje_unutar_seta(df6_2)
df6_3=skaliranje_unutar_seta(df6_3)
df6_4=skaliranje_unutar_seta(df6_4)
df6_5=skaliranje_unutar_seta(df6_5)
df6_6=skaliranje_unutar_seta(df6_6)
df6_7=skaliranje_unutar_seta(df6_7)
df6_8=skaliranje_unutar_seta(df6_8)
df6_9=skaliranje_unutar_seta(df6_9)
df6_10=skaliranje_unutar_seta(df6_10)
df6_11=skaliranje_unutar_seta(df6_11)
df6_12=skaliranje_unutar_seta(df6_12)
df6_13=skaliranje_unutar_seta(df6_13)
df6_14=skaliranje_unutar_seta(df6_14)
df6_15=skaliranje_unutar_seta(df6_15)


df7_1=skaliranje_unutar_seta(df7_1)
df7_2=skaliranje_unutar_seta(df7_2)
df7_3=skaliranje_unutar_seta(df7_3)
df7_4=skaliranje_unutar_seta(df7_4)
df7_5=skaliranje_unutar_seta(df7_5)
df7_6=skaliranje_unutar_seta(df7_6)
df7_7=skaliranje_unutar_seta(df7_7)
df7_8=skaliranje_unutar_seta(df7_8)
df7_9=skaliranje_unutar_seta(df7_9)
df7_10=skaliranje_unutar_seta(df7_10)
df7_11=skaliranje_unutar_seta(df7_11)
df7_12=skaliranje_unutar_seta(df7_12)
df7_13=skaliranje_unutar_seta(df7_13)



df8_1=skaliranje_unutar_seta(df8_1)
df8_2=skaliranje_unutar_seta(df8_2)
df8_3=skaliranje_unutar_seta(df8_3)
df8_4=skaliranje_unutar_seta(df8_4)
df8_5=skaliranje_unutar_seta(df8_5)
df8_6=skaliranje_unutar_seta(df8_6)
df8_7=skaliranje_unutar_seta(df8_7)

# Fit

In [ ]:
autokor = Model(FCS_autokorelacija)

test_set=[df3_2,df6_5,df6_7,df6_10,df7_8,df7_9,df8_6]

#difuzijske konstante test seta
D_df3_2=2.82E-10
D_df3_5=1.16E-10
D_df3_7=5.25E-11
D_df6_10=2.67E-11
D_df7_8=4.05E-11
D_df7_9=3.24E-11
D_df8_6=9.85E-11


D_true=[D_df3_2,D_df3_5,D_df3_7,D_df6_10,D_df7_8,D_df7_9,D_df8_6]

#r_0 detekcijskog volumena
r_0=4.07E-7

#pomačno vrijeme
x=np.array(mjerenje(1,"19.9 difuzija")["Time[µs]"])

#
D_lista_test,M_D_lista_test,D_razlika=fit_test(test_set,D_true)

# Označavanje, pivotiranje i kreiranje train, valid i test seta

In [ ]:
#difuzijske konstante

df3_1.loc["D"]=3.03E-6
df3_2.loc["D"]=2.82E-6
df3_3.loc["D"]=2.25E-6
df3_4.loc["D"]=1.57E-6
df3_5.loc["D"]=1.16E-6
df3_6.loc["D"]=7.61E-7
df3_7.loc["D"]=5.25E-7
df3_8.loc["D"]=4.34E-7
df3_9.loc["D"]=3.46E-7
df3_10.loc["D"]=2.86E-7
df3_11.loc["D"]=2.50E-7
df3_12.loc["D"]=2.14E-7
df3_13.loc["D"]=1.86E-7
df3_14.loc["D"]=1.58E-7
df3_15.loc["D"]=6.29E-7
df3_16.loc["D"]=9.93E-7
df3_17.loc["D"]=4.39E-7
df3_18.loc["D"]=2.18E-7

df5_20.loc["D"]=2.82E-6 
df5_21.loc["D"]=2.21E-6
df5_22.loc["D"]=1.54E-6
df5_23.loc["D"]=1.14E-6

df6_1.loc["D"]=2.89E-6
df6_2.loc["D"]=2.67E-6
df6_3.loc["D"]=2.14E-6
df6_4.loc["D"]=1.48E-6
df6_5.loc["D"]=1.10E-6
df6_6.loc["D"]=1.10E-6
df6_7.loc["D"]=4.93E-7
df6_8.loc["D"]=4.05E-7
df6_9.loc["D"]=3.23E-7
df6_10.loc["D"]=2.67E-7
df6_11.loc["D"]=2.33E-7
df6_12.loc["D"]=2.01E-7
df6_13.loc["D"]=1.73E-7
df6_14.loc["D"]=1.43E-7
df6_15.loc["D"]=9.42E-7



df7_1.loc["D"]=2.12E-6
df7_2.loc["D"]=1.64E-7
df7_3.loc["D"]=1.40E-7
df7_4.loc["D"]=2.61E-7
df7_5.loc["D"]=2.29E-7
df7_6.loc["D"]=1.96E-7
df7_7.loc["D"]=4.95E-7
df7_8.loc["D"]=4.05E-7
df7_9.loc["D"]=3.24E-7
df7_10.loc["D"]=9.39E-7
df7_11.loc["D"]=1.12E-6
df7_12.loc["D"]=1.53E-6
df7_13.loc["D"]=2.99E-6

df8_1.loc["D"]=3.07E-6
df8_2.loc["D"]=1.16E-6
df8_3.loc["D"]=1.83E-7
df8_4.loc["D"]=5.25E-7
df8_5.loc["D"]=9.81E-7
df8_6.loc["D"]=9.85E-7
df8_7.loc["D"]=1.60E-7


df3_1=pivot(df3_1)
df3_2=pivot(df3_2)
df3_3=pivot(df3_3)
df3_4=pivot(df3_4)
df3_5=pivot(df3_5)
df3_6=pivot(df3_6)
df3_7=pivot(df3_7)
df3_8=pivot(df3_8)
df3_9=pivot(df3_9)
df3_10=pivot(df3_10)
df3_11=pivot(df3_11)
df3_12=pivot(df3_12)
df3_13=pivot(df3_13)
df3_14=pivot(df3_14)
df3_15=pivot(df3_15)
df3_16=pivot(df3_16)
df3_17=pivot(df3_17)
df3_18=pivot(df3_18)

df5_20=pivot(df5_20)
df5_21=pivot(df5_21)
df5_22=pivot(df5_22)
df5_23=pivot(df5_23)

df6_1=pivot(df6_1)
df6_2=pivot(df6_2)
df6_3=pivot(df6_3)
df6_4=pivot(df6_4)
df6_5=pivot(df6_5)
df6_6=pivot(df6_6)
df6_7=pivot(df6_7)
df6_8=pivot(df6_8)
df6_9=pivot(df6_9)
df6_10=pivot(df6_10)
df6_11=pivot(df6_11)
df6_12=pivot(df6_12)
df6_13=pivot(df6_13)
df6_14=pivot(df6_14)
df6_15=pivot(df6_15)

df7_1=pivot(df7_1)
df7_2=pivot(df7_2)
df7_3=pivot(df7_3)
df7_4=pivot(df7_4)
df7_5=pivot(df7_5)
df7_6=pivot(df7_6)
df7_7=pivot(df7_7)
df7_8=pivot(df7_8)
df7_9=pivot(df7_9)
df7_10=pivot(df7_10)
df7_11=pivot(df7_11)
df7_12=pivot(df7_12)
df7_13=pivot(df7_13)

df8_1=pivot(df8_1)
df8_2=pivot(df8_2)
df8_3=pivot(df8_3)
df8_4=pivot(df8_4)
df8_5=pivot(df8_5)
df8_6=pivot(df8_6)
df8_7=pivot(df8_7)




In [ ]:

# Kreiranje testnog seta te drugog iz kojega se kasnije kreiraju train i validacijski set
df=df3_1.append([df3_3,df3_4,df3_5,df3_6,df3_7,df3_8,df3_9,df3_10,df3_11,df3_12,df3_14,df3_15,df3_16,df3_17,df3_18,df5_20,df5_21,df5_22,df5_23,df6_1,df6_2,df6_3,df6_4,df6_6,df6_8,df6_9,df6_11,df6_12,df6_13,df6_14,df6_15,df7_1,df7_2,df7_3,df7_4,df7_5,df7_6,df7_7,df7_10,df7_11,df7_12,df7_13,df8_1,df8_2,df8_3,df8_4,df8_5,df8_7])
test=df3_2.append([df6_5,df6_7,df6_10,df7_8,df7_9,df8_6])


#Potrebno je D reskalirati jer inače mreža ne uči najbolje.
df["D"]=df["D"]*1000000
test["D"]=test["D"]*1000000
train,valid=train_test_split(df,test_size=0.15,random_state=random.randint(1,1000),shuffle=True)

# Rezultati s više modela

In [ ]:
#početni indeksi prvog mjerenja svakog podskupa u test setu
granice=[0,14,28,42,56,70,83,97]
broj_runova=10

NN_mean,NN_std=run_vise_modela(broj_runova)


sum1=sum([NN_mean[i,:]/(NN_std[i,:]**2) for i in range(broj_runova)])
sum2=sum([1/(NN_std[i,:]**2) for i in range(broj_runova)])

opca_srednja_vrijednost=sum1/sum2
opca_srednja_vrijednost=opca_srednja_vrijednost*10**(-10)

opca_nepoudanost=np.sqrt(1/sum2)
opca_nepoudanost=opca_nepoudanost*10**(-10)


#D_list_test i M_D_lista_test su srednje vrijednosti i nepouzdanosti dobivene fitom
#D_true i M_D_true su srednje vrijednosti i nepouzdanosti dobivene Stokes-Einstein jednadžbom izvan ovog programa

D_true=[2.82e-10, 1.16e-10, 5.25e-11, 2.67e-11, 4.05e-11, 3.24e-11, 9.85e-11]
M_D_true=[7.53E-12,2.94792E-12,1.31986E-12,7.15717E-13,1.08534E-12,8.68565E-13,2.60307E-12]


In [ ]:
fig, (ax1,ax2, ax3,ax4) = plt.subplots(nrows=1, ncols=4, sharex=True,figsize=(60, 26.5))



linew=10
marks=30

ax1.errorbar(x=1,y=D_lista_test[2],yerr=M_D_lista_test[2],fmt='o',linewidth=linew,label="$D_{FIT}$",markersize=marks)#capsize=1
ax1.errorbar(x=1,y=opca_srednja_vrijednost[2],yerr=opca_nepoudanost[2],fmt='o',linewidth=linew,label="$D_{NN}$",markersize=marks)#capsize=1
ax1.errorbar(x=1,y=D_true[2],yerr=M_D_true[2],fmt='o',linewidth=linew,label="$D_{S-E}$",markersize=marks)#capsize=1

ax2.errorbar(x=1,y=D_lista_test[3],yerr=M_D_lista_test[3],fmt='o',linewidth=linew,label="$D_{FIT}$",markersize=marks)#capsize=1
ax2.errorbar(x=1,y=opca_srednja_vrijednost[3],yerr=opca_nepoudanost[3],fmt='o',linewidth=linew,label="$D_{NN}$",markersize=marks)#capsize=1
ax2.errorbar(x=1,y=D_true[3],yerr=M_D_true[3],fmt='o',linewidth=linew,label="$D_{S-E}$",markersize=marks)#capsize=1

ax3.errorbar(x=1,y=D_lista_test[4],yerr=M_D_lista_test[4],fmt='o',linewidth=linew,label="$D_{FIT}$",markersize=marks)#capsize=1
ax3.errorbar(x=1,y=opca_srednja_vrijednost[4],yerr=opca_nepoudanost[4],fmt='o',linewidth=linew,label="$D_{NN}$",markersize=marks)#capsize=1
ax3.errorbar(x=1,y=D_true[4],yerr=M_D_true[4],fmt='o',linewidth=linew,label="$D_{S-E}$",markersize=marks)#capsize=1


ax4.errorbar(x=1,y=D_lista_test[5],yerr=M_D_lista_test[5],fmt='o',linewidth=linew,label="$D_{FIT}$",markersize=marks)#capsize=1
ax4.errorbar(x=1,y=opca_srednja_vrijednost[5],yerr=1.5*opca_nepoudanost[5],fmt='o',linewidth=linew,label="$D_{NN}$",markersize=marks)#capsize=1
ax4.errorbar(x=1,y=D_true[5],yerr=M_D_true[5],fmt='o',linewidth=linew,label="$D_{S-E}$",markersize=marks)#capsize=1

lab_size=50
legend_size=45

ax1.tick_params(labelsize=lab_size)
ax2.tick_params(labelsize=lab_size)
ax3.tick_params(labelsize=lab_size)
ax4.tick_params(labelsize=lab_size)


ax1.legend(prop={'size': legend_size},title=0)
ax2.legend(prop={'size': legend_size},title=0)
ax3.legend(prop={'size': legend_size},title=0)
ax4.legend(prop={'size': legend_size},title=0)


ax1.yaxis.get_offset_text().set_fontsize(lab_size)
ax2.yaxis.get_offset_text().set_fontsize(lab_size)
ax3.yaxis.get_offset_text().set_fontsize(lab_size)
ax4.yaxis.get_offset_text().set_fontsize(lab_size)
ax1.ticklabel_format(axis="y",useMathText=True)
ax2.ticklabel_format(axis="y",useMathText=True)
ax3.ticklabel_format(axis="y",useMathText=True)
ax4.ticklabel_format(axis="y",useMathText=True)

ax1.set_ylabel("D [$\dfrac{m^{2}}{s}$]",fontsize=70)


fig, (ax1, ax3,ax4) = plt.subplots(nrows=1, ncols=3, sharex=True,figsize=(60, 27))



ax1.errorbar(x=1,y=D_lista_test[0],yerr=M_D_lista_test[0],fmt='o',linewidth=linew,label="$D_{FIT}$",markersize=marks)#capsize=1
ax1.errorbar(x=1,y=opca_srednja_vrijednost[0],yerr=opca_nepoudanost[0],fmt='o',linewidth=linew,label="$D_{NN}$",markersize=marks)#capsize=1
ax1.errorbar(x=1,y=D_true[0],yerr=M_D_true[0],fmt='o',linewidth=linew,label="$D_{S-E}$",markersize=marks)#capsize=1

ax3.errorbar(x=1,y=D_lista_test[6],yerr=M_D_lista_test[6],fmt='o',linewidth=linew,label="$D_{FIT}$",markersize=marks)#capsize=1
ax3.errorbar(x=1,y=opca_srednja_vrijednost[6],yerr=opca_nepoudanost[6],fmt='o',linewidth=linew,label="$D_{NN}$",markersize=marks)#capsize=1
ax3.errorbar(x=1,y=D_true[6],yerr=M_D_true[6],fmt='o',linewidth=linew,label="$D_{S-E}$",markersize=marks)#capsize=1

ax4.errorbar(x=1,y=D_lista_test[1],yerr=M_D_lista_test[1],fmt='o',linewidth=linew,label="$D_{FIT}$",markersize=marks)#capsize=1
ax4.errorbar(x=1,y=opca_srednja_vrijednost[1],yerr=opca_nepoudanost[1],fmt='o',linewidth=linew,label="$D_{NN}$",markersize=marks)#capsize=1
ax4.errorbar(x=1,y=D_true[1],yerr=M_D_true[1],fmt='o',linewidth=linew,label="$D_{S-E}$",markersize=marks)#capsize=1


ax1.tick_params(labelsize=lab_size)
ax3.tick_params(labelsize=lab_size)
ax4.tick_params(labelsize=lab_size)


ax1.legend(prop={'size': legend_size},title=0)
ax3.legend(prop={'size': legend_size},title=0)
ax4.legend(prop={'size': legend_size},title=0)



ax1.yaxis.get_offset_text().set_fontsize(lab_size)
ax3.yaxis.get_offset_text().set_fontsize(lab_size)
ax4.yaxis.get_offset_text().set_fontsize(lab_size)


ax1.ticklabel_format(axis="y",useMathText=True)
ax3.ticklabel_format(axis="y",useMathText=True)
ax4.ticklabel_format(axis="y",useMathText=True)

ax1.set_ylabel("D [$\dfrac{m^{2}}{s}$]",fontsize=70)
#plt.savefig("error_barovi.png")